# Job Posting Analysis (DeepSeek)
---

## The Case

[PyFi](https://pyfi.com) teaches finance professionals how to code in Python. To support our marketing, we analyzed a set of job postings to understand why companies hire finance professionals with Python skills.

As part of our analysis, we asked, "What alternative tools do these job postings list alongside Python?" Our research assistant copied tools into a spreadsheet, but the tool names and list formatting were inconsistent. In order to analyze this unstructured data, we needed to convert it into structured data.

The purpose of this Python program is to automatically convert the unstructured data in the "Alternatives" column of Job Data.csv into a set of correctly-flagged indicator columns to support PyFi's analysis.

This version of the program uses the [DeepSeek API](https://api-docs.deepseek.com/) (which we access through the OpenAI package).

<br>

## Step 1: Preparation

First, we prepare to analyze the job posting data by importing the necessary packages, reading the job posting data into our program, and defining a prompting function.

In [ ]:
# Installing the OpenAI package so that it is available for import

%pip install openai

In [ ]:
# Importing packages

import pandas as pd
from openai import OpenAI
import json

In [ ]:
# Reading the job posting data into the program

posting_data = pd.read_csv('Job Data.csv')
posting_data

In [ ]:
# Creating an OpenAI-type client object with a DeepSeek API key and base URL

client = OpenAI(api_key = 'Your API Key', base_url = 'https://api.deepseek.com')


# Defining a function to prompt models through the DeepSeek API

def prompt(model, user_prompt, system_prompt = None, response_format = None):
    try:
        
        if system_prompt == None:
            prompt_list = [
                {'role' : 'user', 'content' : user_prompt}
            ]
        else:
            prompt_list = [
                    {'role' : 'system', 'content' : system_prompt},
                    {'role' : 'user', 'content' : user_prompt}
            ]
    
        if response_format == None:
            model_response = client.chat.completions.create(
                model = model,
                messages = prompt_list
            )
            function_output = model_response.choices[0].message.content
        elif response_format == 'json_object':
            model_response = client.chat.completions.create(
                model = model,
                messages = prompt_list,
                response_format = {'type' : response_format}
            )
            function_output = json.loads(model_response.choices[0].message.content)
        else:
            raise ValueError("response_format must be None or 'json_object'")
        
        return function_output
    
    except Exception as e:
        return f'An error occurred: {e}'

<br>

## Step 2: Extracting a List of Unique Tools

Next, we extract a list of unique tools from the Alternatives column with the DeepSeek API. To reduce error, we divide this task into three prompts:

1. Extract a list of unique tools.
2. Rename the tools according to Python variable syntax.
3. Organize the tools into a Python list.

In [ ]:
# Isolating the Alternatives column and converting its data into a string

alternatives_string = posting_data['Alternatives'].to_string(index = False)

In [ ]:
# PROMPT 1
# Extracting an unstructured list of unique tools from the Alternatives data with R1

user_prompt_1 = f'The following data contains tools which can be used instead of or alongside Python. Give me a list of unique tools. Exclude Python.\n{alternatives_string}'

unstructured_tool_list = prompt('deepseek-reasoner', user_prompt_1)
unstructured_tool_list

In [ ]:
# PROMPT 2
# Renaming the tools with R1

user_prompt_2 = f'The following text contains a list of tools which can be used instead of or alongside Python. Format the name of each tool so that it can serve as the name of a variable in a Python program. No explanation is necessary.\n{unstructured_tool_list}'

unstructured_tool_list_renamed = prompt('deepseek-reasoner', user_prompt_2)
unstructured_tool_list_renamed

In [ ]:
# PROMPT 3
# Organizing the tools into a Python list with V3

user_prompt_3 = unstructured_tool_list_renamed
system_prompt_3 = '''
The user will provide a list of tools which can be used instead of or alongside Python. Return the list to the user in JSON format according to the following example:

EXAMPLE INPUT:
Aladdin, Alteryx, amazon_quicksight

EXAMPLE OUTPUT:
{
    "tools": ["Aladdin", "Alteryx", "amazon_quicksight"]
}
'''
structured_tool_list = prompt('deepseek-chat', user_prompt_3, system_prompt_3, response_format ='json_object')['tools']


# Displaying the result

print(f'This list contains {len(structured_tool_list)} elements:')
structured_tool_list.sort()
structured_tool_list

<br>

## Step 3: Creating Unflagged Indicator Columns

Next, we make any necessary manual changes to the list of tools and then use it to create a set of unflagged indicator columns.

In [ ]:
# Manually cleaning the list
# (Because the output of the prior cell varies, the code in this cell may change.)

structured_tool_list.remove('amazon_quicksight')    # To prevent confusion with Amazon Web Services
structured_tool_list.append('quicksight')
structured_tool_list.sort()
structured_tool_list

In [ ]:
# Adding indicator columns to the DataFrame

posting_data[structured_tool_list] = False
posting_data.columns

<br>

## Step 4: Flagging the Indicator Columns

Now that we have a list of unique tools and a set of unflagged indicator columns, we can loop through each row in the Alternatives column, identify listed tools, and update the indicator columns accordingly.

This step sends a unique prompt to the DeepSeek model for each row, directing it to identify both expected and unexpected tools (in case the earlier model missed something in its response to Prompt #1).

If the model identifies an expected tool in a row of the Alternatives column, then this step will flag the corresponding indicator column.

Because the dataset contains 100 rows, this step will take longer to compute than the others.

In [ ]:
# PROMPT 4
# Analyzing each row of alternatives and updating indicator columns accordingly

# Initializing a dictionary for documenting unexpected tools

unexpected_tools_dict = {}


# Looping through each row of Alternatives, updating indicator columns, and recording
# unexpected tools.

for index in range(len(posting_data)):

    # Specify data for analysis
    row_data = str(posting_data['Alternatives'][index])
    
    # Define prompts
    user_prompt_4 = row_data
    system_prompt_4 = f'''The user will provide some number of tools (possibly zero) which can be used instead of or alongside Python. Although there may be some spelling differences, I expect that any tools in the row data will be members of the following set:
    
    Expected Tool Set: {str(structured_tool_list)}
    
    However, it is possible that the user's data could contain other tools. Indicate the presence of expected tools by adding the name of the tool to the expected_tools field in the supplied JSON schema. Indicate the presence of unexpected tools by adding the name of the tool to the unexpected_tools field in the supplied JSON schema. 
    
    If an expected tool is present in the user's data, return the name of the tool exactly as it is written in the Expected Tool Set (including identical capitalization), even if the name of the tool is slightly different in the user's data.

    EXAMPLE INPUT:
    SQL, R, C++, DAX, or other analytical languages

    EXAMPLE OUTPUT:
    {{
        "expected_tools": ["sql", "r", "c_plus_plus"],
        "unexpected_tools": ["DAX"]
    }}
    '''
    # Prompt V3
    row_tools_object = prompt('deepseek-chat', user_prompt_4, system_prompt_4, response_format = 'json_object')
    print(row_tools_object)

    # Update indicator columns
    for tool in row_tools_object['expected_tools']:
        posting_data.at[index, tool] = True

    # Document unexpected tools        
    if len(row_tools_object['unexpected_tools']) > 0:
        unexpected_tools_dict[index] = row_tools_object['unexpected_tools']

<br>

## Step 5: Evaluating Results

Finally, we display the model's collection of unexpected tools and output the modified DataFrame as a CSV to feed into our scoring tool.

In [ ]:
# Displaying the model's collection of unexpected tools

print(f'The model identified unexpected Tools in {len(unexpected_tools_dict)} rows:\n{unexpected_tools_dict}\n---')

In [ ]:
# Exporting the modified DataFrame as a CSV for further evaluation

posting_data.to_csv('DeepSeek Test Results.csv')

<br>

---

*If you are a finance professional and would like to learn Python, check out PyFi's introductory courses at [PyFi.com](https://pyfi.com).*